In [10]:
import os
import re
import requests
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd

In [11]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [12]:
kam = 'data_raw/nobelistvo'

In [13]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [14]:
koho = pd.read_json(os.path.join('data_raw','wikidata_nobelovky_pro_filtr.json'))

In [15]:
sloupce = ['100','245','700']

In [16]:
df = pd.read_parquet(os.path.join('data/cnb_sloupce','008.parquet'))
for s in sloupce:
    df = df.merge(pd.read_parquet(os.path.join('data/cnb_sloupce',f'{s}.parquet')), left_index=True, right_index=True, how='outer')

In [17]:
df.sample(10)

,008,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,245_v,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_n,700_k,700_m,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_g
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
bk196404835,[971112s1964 xr cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Polygrafický průmysl :],"[Seznam knih, které vydalo SNTL v letech 1953-...",None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk19990769252,[991104s1999 xr e 000 macze],[1],"[Pribram, Karl H.,]",[xx0003841],[aut],[1919-2015],None,None,None,None,[1],[0],[Mozek a mysl :],[holonomní pohled na svět /],[Karl H. Pribram ; uspořádal Jiří Fiala ; [z a...,None,None,None,None,None,None,"[1, 1, 1]","[Fiala, Jiří,, Adamovič, Jiří, Ladzianska, Zuz...","[trl, trl, trl]","[1939-2012, None, None]","[jn19981001074, xx0064518, None]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
hud04701060,[081223s1944 xr sgz g nn cze],[1],"[Maršálek, O.]",None,[cmp],None,None,None,None,None,[1],[0],[Krysařova píšťalka],[fox /],[slova Lambert J. Binhack ; hudba O. Maršálek],None,None,[[hudebnina] :],None,None,None,[1],"[Binhack, Lambert J.]",[lyr],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
cpg20102030510,[100125s2009 xr nnn g kncze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Ctihodný občan],"[jak zastavit vraha, který už je za mřížemi? :...",None,None,None,[[grafika] :],None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk20152720135,[980928d19621974xr x | b0cze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[0],[0],[Zahraniční zdravotnické normy],None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk195102651,[980311s1951 xr 1 cze],[1],"[Wedding, Alex,]",[xx0005615],[aut],[1905-1966],None,None,None,None,[1],[0],[Prapor svobody :],[Hist. román o selských bouřích v Německu /],[Alex Wedding ; Z něm. orig. Die Fahne des Pfe...,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20213367788,[211020s2021 xr ach e 000 0ccze],[1],"[Miketa, Kamil,]",[jo20000073768],[aut],[1969-],None,None,None,None,[1],[0],[Podnikatelské legendy první republiky a jejic...,None,[Kamil Miketa],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ck9100674,[910221s1990 xr a u0|0 | cze],[1],"[Martinec, Petr,]",[jo2002094094],[aut],[1942-],None,None,None,None,[1],[0],[Pokryvné útvary karbonu čs. části hornoslezsk...,None,"[Petr Martinec, Jan Krajíček]",None,None,None,None,None,None,[1],"[Krajíček, Jan]",[aut],None,[jx20071203064],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
bk197000188,[970422s1969 xr cze],[1],"[Koutský, Jaroslav,]",[jk01062397],[aut],[1929-2016],None,None,None,None,[1],[0],[Materiálové vady ocelových nástrojů /],None,"[[Autoři:] Jaroslav Koutský, František Jandoš]",None,None,None,None,None,None,[1],"[Jandoš, František]",[aut],None,[xx0054906],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [18]:
for index, row in koho.iterrows():
    nazev_souboru = f"nobelovka_{row['nobelovka']}_{row['100_7']}.parquet"
    if nazev_souboru not in os.listdir(kam):
        print(row['nobelovka'])
        df_slice = df[df['100_7'].astype(str).str.contains(row['100_7'], na=False) | df['700_7'].astype(str).str.contains(row['100_7'], na=False)]
        df_slice = df_slice.explode('008')
        df_slice['rok'] = df_slice['008'].apply(lambda x: najdi_rok(x))
        df_slice['nobelovka'] = row['nobelovka']
        df_slice['titul'] = df_slice['245_a'].apply(lambda x: x[0].replace(' /','').replace('=','').replace(' :','').replace(';','').strip() if x != None else None)
        df_slice.to_parquet(os.path.join(kam, nazev_souboru))

2024
